<a href="https://colab.research.google.com/github/wilmarpc/Celonis-ML-Python/blob/main/Utilidades/Obtener_Tablas_Extraccion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --extra-index-url=https://pypi.celonis.cloud/ pycelonis
import pycelonis
url = "https://empresas-cmpc.us-1.celonis.cloud/"
# Application Key:
api_token = "MTlkYzJhZmEtNDc5Mi00YTM3LTg5MjEtNTY0YjhkMDA1YzFjOnJQSExxT0RzWENoUm5ITEJwV2FZcmVjOGNnbDF0NDM4d1lWa1FTc3ZsYTdT"
key_type = "APP_KEY"

from pycelonis import get_celonis
celonis = get_celonis(base_url=url, api_token=api_token, key_type=key_type)

import pandas as pd
from pycelonis.pql import PQL, PQLColumn, PQLFilter, OrderByColumn

import os
from pycelonis.ems import ExportType

# !pip install gocept.pseudonymize
#import gocept.pseudonymize
#gocept.pseudonymize.name('Vimladil', 'secret')
#gocept.pseudonymize.email('mail@gocept.com', 'secret')

In [2]:
data_pool_id = "49899c1e-b116-40c2-8c81-7ab8009f0c6c"
data_pool = celonis.data_integration.get_data_pool(data_pool_id)
data_pool

DataPool(id='49899c1e-b116-40c2-8c81-7ab8009f0c6c', name='SAP ECC - Accounts Receivable')

In [3]:
data_model = data_pool.get_data_model("03cd113f-5e9e-415d-b112-870dc5838f4c")
data_model

DataModel(id='03cd113f-5e9e-415d-b112-870dc5838f4c', name='SAP ECC - Accounts Receivable Data Model (columns reduction)', pool_id='49899c1e-b116-40c2-8c81-7ab8009f0c6c')

"""
source_data_job = source_data_pool.data_jobs.find("Name or ID of the source data job.")
source_extraction = source_data_job.extractions.ids['ID of the source extraction task.']
https://empresas-cmpc.us-1.celonis.cloud/integration/ui/
pools/7edb34cd-1802-4a1c-9ee8-53acf70f3485/data-configuration/
data-jobs/e183e9d5-390a-4127-b644-9a28506d3ff1/
extractions/e24489cb-bc5d-42b8-a032-be607f822e79
tableId=244736e1-e205-4632-b405-ea416aa3b23e&tab=extraction

id='e183e9d5-390a-4127-b644-9a28506d3ff1' name='SAP ECC - VIM Connection' data_pool_id='7edb34cd-1802-4a1c-9ee8-53acf70f3485' data_source_id='059dd70f-de9b-41e7-bb52-5068a97b47b5' time_stamp=datetime.datetime(2022, 12, 13, 16, 31, 24, 557000, tzinfo=datetime.timezone.utc) status=<ExecutionStatus.SUCCESS: 'SUCCESS'> current_execution_id=None dag_based_execution_enabled=True latest_execution_item_id='cf1c7d95-c698-42f3-aff3-af414300d628' client=<pycelonis_core.client.client.Client object at 0x7ff27154fdc0>
[
	Extraction(id='e24489cb-bc5d-42b8-a032-be607f822e79', pool_id='7edb34cd-1802-4a1c-9ee8-53acf70f3485', task_id='e91bf75b-ea2e-4bfa-9ebe-816125f2829d', task_type=<TaskType.EXTRACTION: 'EXTRACTION'>, name='Extraction: VIM Connection', job_id='e183e9d5-390a-4127-b644-9a28506d3ff1', disabled=False)
]
source_data_connection = data_pool.get_data_connection("059dd70f-de9b-41e7-bb52-5068a97b47b5")

source_data_job = data_pool.get_job("e183e9d5-390a-4127-b644-9a28506d3ff1")
source_extraction = source_data_job.get_task('e91bf75b-ea2e-4bfa-9ebe-816125f2829d').

print(source_data_job)
print(source_extraction)
"""

In [8]:
def extract_table_from_data_pool(
    celonis_object: pycelonis.celonis.Celonis,
    data_pool: pycelonis.ems.data_integration.data_pool.DataPool,
    data_model: pycelonis.ems.data_integration.data_model.DataModel,
    table: pycelonis.ems.data_integration.data_pool_table.DataPoolTable
    #table: pycelonis.ems.data_integration.data_model.DataModelTable
    ) -> pd.DataFrame:

    """This function creates the PQL query to extract a whole
    table from a data model. If the table is too big, the kernel might
    shut down, make sure the RAM memory is enough.

        Input:
            celonis_object: Celonis object already instantiated
            data_pool: Data Pool object already instantiated
            data_model: Data Model object already instantiated
            table: Table object already instantiated

        Returns:
            df: DataFrame containing the target table.
    """

    # Instantiate query object
    query = PQL(distinct=False, limit=None, offset=None)
     
    tabla = table.alias if str(table.alias) != 'None' else table.name

    # Populate query with all the columns from the table
    for PQL_column in table.get_columns():
      if PQL_column.name != '_CELONIS_CHANGE_DATE':
        if ((table.name == '/OPT/VIM_1HEAD' and PQL_column.name == 'ARCHIVE_USER') or
           (table.name == '/OPT/VIM_1LOG' and PQL_column.name == 'ACTUAL_AGENT') or
           (table.name == '/ORS/APPR_LOG' and PQL_column.name == 'ACTUAL_APPR') or
           (table.name == 'BKPF' and PQL_column.name == 'USNAM') or
           (table.name == 'AR_BKPF' and PQL_column.name == 'USNAM') or
           (table.name == 'AR_BKPF_CLEARING' and PQL_column.name == 'USNAM') or
           (table.name == 'JCDS' and PQL_column.name == 'USNAM') or
           (table.name == 'MKPF' and PQL_column.name == 'USNAM') or
           (table.name == 'NAST' and PQL_column.name == 'USNAM') or
           (table.name == 'CDHDR' and PQL_column.name == 'USERNAME') or
           (table.name == 'LFA1' and PQL_column.name == 'NAME1') or
           (table.name == 'KNA1' and PQL_column.name == 'NAME1') or
           (table.name == 'T001W' and PQL_column.name == 'NAME1') or
           (table.name == 'ADR6' and PQL_column.name == 'SMTP_ADDR') or
           (table.name == 'BKPF_BSAK' and PQL_column.name == 'USNAM') or
           (table.name == 'BKPF_BSIK' and PQL_column.name == 'USNAM') or
           (table.name == 'EKBE' and PQL_column.name == 'ERNAM') or
           (table.name == 'EKKO' and PQL_column.name == 'ERNAM') or
           (table.name == 'LIKP' and PQL_column.name == 'ERNAM') or
           (table.name == 'LIPS' and PQL_column.name == 'ERNAM') or
           (table.name == 'VBAK' and PQL_column.name == 'ERNAM') or
           (table.name == 'VBAP' and PQL_column.name == 'ERNAM') or
           (table.name == 'VBRP' and PQL_column.name == 'ERNAM') or
           (table.name == 'VTTK' and PQL_column.name == 'ERNAM') or
           (table.name == 'VTTP' and PQL_column.name == 'ERNAM') or
           (table.name == 'RBKP' and PQL_column.name == 'USNAM') or
           (table.name == 'T024' and PQL_column.name == 'SMTP_ADDR') or
           (table.name == 'T014T' and PQL_column.name == 'KKBTX') or
           (table.name == 'TVSTT' and PQL_column.name == 'VTEXT') or
           (table.name == '_CEL_O2C_ACTIVITIES' and PQL_column.name == 'USER_NAME') or
           (table.name == 'O2C_KNA1' and PQL_column.name == 'NAME1') or
           (table.name == 'O2C_LIPS' and PQL_column.name == 'ERNAM') or
           (table.name == 'O2C_NAST' and PQL_column.name == 'USNAM') or
           (table.name == 'O2C_VBAK' and PQL_column.name == 'ERNAM') or
           (table.name == 'O2C_VBAK' and PQL_column.name == 'VKORG_TEXT') or
           (table.name == 'O2C_VBAK' and PQL_column.name == 'KKBER_TEXT') or
           (table.name == 'O2C_VBAK' and PQL_column.name == 'BUKRS_TEXT') or
           (table.name == 'AR_BKPF' and PQL_column.name == 'BUKRS_TEXT') or
           (table.name == 'AR_BSEG' and PQL_column.name == 'BUKRS_TEXT') or
           (table.name == 'AR_BSEG_CLEARING' and PQL_column.name == 'BUKRS_TEXT') or
           (table.name == 'AR_BSEG_CLEARING' and PQL_column.name == 'VNAME') or
           (table.name == 'AR_BKPF_CLEARING' and PQL_column.name == 'BUKRS_TEXT') or
           (table.name == 'O2C_VBAP' and PQL_column.name == 'ARKTX') or
           (table.name == 'O2C_VBAP' and PQL_column.name == 'ERNAM') or
           (table.name == 'O2C_VBAP' and PQL_column.name == 'MATKL_TEXT') or
           (table.name == 'O2C_VBAP' and PQL_column.name == 'WERKS_TEXT') or
           (table.name == 'O2C_VBAP' and PQL_column.name == 'LTEXT') or
           (table.name == 'O2C_VBRP' and PQL_column.name == 'ERNAM') or
           (table.name == 'V_P2P_O2C' and PQL_column.name == 'CLIEN') or
           (table.name == 'V_P2P_O2C' and PQL_column.name == 'PROVE') or
           (table.name == 'V_VTTP' and PQL_column.name == 'ERNAM') or
           (table.name == 'P2P_EKBE' and PQL_column.name == 'ERNAM') or
           (table.name == 'P2P_EKKO' and PQL_column.name == 'ERNAM') or
           (table.name == 'P2P_EKKO' and PQL_column.name == 'BUKRS_TEXT') or
           (table.name == 'P2P_EBAN' and PQL_column.name == 'ERNAM') or
           (table.name == 'P2P_EINE' and PQL_column.name == 'EKORG_TEXT') or
           (table.name == 'P2P_EINE' and PQL_column.name == 'EKGRP_TEXT') or
           (table.name == 'P2P_QALS' and PQL_column.name == 'ERSTELLER') or
           (table.name == 'AR_BSEG' and PQL_column.name == 'VNAME') or
           (table.name == 'P2P_QAVE' and PQL_column.name == 'VNAME') or
           (table.name == 'P2P_NAST' and PQL_column.name == 'USNAM') or
           (table.name == 'P2P_BKPF' and PQL_column.name == 'USNAM') or
           (table.name == 'P2P_CONTRACT_INFO' and PQL_column.name == 'NAME1') or
           (table.name == 'P2P_CONTRACT_INFO' and PQL_column.name == 'EKORG_TEXT') or
           (table.name == 'AR_KNA1' and PQL_column.name == 'NAME1') or
           (table.name == 'P2P_LFA1' and PQL_column.name == 'NAME1') or
           (table.name == 'P2P_EKPO' and PQL_column.name == 'BUKRS_TEXT	') or
           (table.name == 'P2P_EKPO' and PQL_column.name == 'WERKS_TEXT') or
           (table.name == '_CEL_P2P_ACTIVITIES' and PQL_column.name == 'USER_NAME') or
           (table.name == '_CEL_AR_ACTIVITIES' and PQL_column.name == 'USER_NAME') or
           (table.name == '' and PQL_column.name == '') or
           (table.name == 'USR21' and PQL_column.name == 'BNAME') or
           (table.name == 'T001' and PQL_column.name == 'BUTXT') or
           (table.name == 'USR02' and PQL_column.name == 'BNAME')):
          consulta = f""" STRINGHASH("{tabla}"."{PQL_column.name}") """
        else:
          consulta = f""" "{tabla}"."{PQL_column.name}" """
        
        query += PQLColumn(name=PQL_column.name, query=consulta)

        if (table.name == '/OPT/VIM_1HEAD' or
            table.name == '/OPT/VIM_1ITEM' or
            table.name == '/OPT/VIM_1LOG'):
          orden = f""" "{tabla}"."DOCID" """
        elif (table.name == 'BKPF_BSAK' or table.name == 'BKPF_BSIK'):
          orden = f""" "{tabla}"."BUDAT" """
        elif table.name == 'EKBE':
          orden = f""" "{tabla}"."CPUDT" """
        elif table.name == 'BKPF':
          orden = f""" "{tabla}"."BLDAT" """
        elif table.name == 'BSEG':
          orden = f""" "{tabla}"."AUGDT" """
        elif table.name == 'RSEG':
          orden = f""" "{tabla}"."BELNR" """
        elif (table.name == 'CDHDR' or
              table.name == 'CDPOS'):
          orden = f""" "{tabla}"."CHANGENR" """
        else:
          orden = None

    if orden is not None:
      query += OrderByColumn(query=orden, ascending=False)

    #print(query)

    try:
        # If you have USE permissions on the data model this method can be used
        df = DataModel.export_data_frame_from(
                                     celonis_object.client,
                                     pool_id=data_model.pool_id,
                                     data_model_id=data_model.id,
                                     query=query)
        print("You have use permissions")
    except:
        # Should work if you don't have USE permissions
        df = data_model.export_data_frame(query)
        print("You might not have USE permissions")
        data_export = data_model.create_data_export(query=query, export_type=ExportType.CSV)
        data_export.wait_for_execution()
        chunks = data_export.get_chunks()
        for chunk in chunks:
          with open(f"{table.name}.csv", "wb") as f:
            f.write(chunk.read())
    return df

In [12]:
for table in data_model.get_tables():
  print(table.name+' '+table.id)
  if (table.name == 'AR_KNA1'):
    tabla = table
    print("tabla: " + tabla.id)
    result_df = extract_table_from_data_pool(celonis, data_pool, data_model, tabla)   

_CEL_AR_ACTIVITIES 676c0285-6191-41ac-9183-f6a023d1c391
AR_KNB5 4dd2d816-7ffa-4f4d-940c-861ff668c66c
AR_KNB1 f9c83036-0631-4536-bab1-bf6005774f48
AR_BKPF b815022f-3c9c-4c02-afb9-ab0298d9cf2d
AR_TCURF 2b0f5b53-8a00-4a11-a495-53fbae4416c3
AR_TCURR 7cc98d87-2d9a-4259-89bc-b7431a1d6312
AR_TCURX 1bf0d9c4-d1f8-47b5-a98b-7004da08a69c
AR_KNA1 31c1c96a-a7c6-4a93-b8a3-b2748861ed36
tabla: 31c1c96a-a7c6-4a93-b8a3-b2748861ed36


0it [00:00, ?it/s]

PyCelonisDataExportFailedError: ignored

In [11]:
tg_url = "https://ofi-partner-sandbox.us-1.celonis.cloud/"
# Application Key:
tg_api_token = "ZThlZjQ3ODMtMjRmMC00OTU5LWJjMjgtZDQ0YTNlMTJiMGE4OlhUUmFKYUxrdjdxcEowSU5OeFh1czhaeXRvMUUrd08wQWoreG5jMmlDeEhI"
tg_key_type = "APP_KEY"

tg_celonis = get_celonis(base_url=tg_url, api_token=tg_api_token, key_type=tg_key_type)

tg_data_pool_id = "0113fe2e-1535-4979-8545-d50af81bc819"
tg_data_pool = tg_celonis.data_integration.get_data_pool(tg_data_pool_id)
tg_data_pool

tg_data_model = tg_data_pool.get_data_model("d5a7d7f4-aef7-4e08-bc4b-4ac209406fef")
tg_data_model

#tg_data_connection = tg_data_pool.get_data_connection("6e4a60b0-70f5-406c-8846-778d2e61bc0f")

for table in data_model.get_tables():
  if (table.name != '_CEL_VIM_ACTIVITIES' and table.name != 'VIM_/OPT/VIM_1ITEM'
      and table.name != 'VBAK' and table.name != 'O2C_VBAK' and table.name != '_CEL_P2P_ACTIVITIES'
      and table.name != '_CEL_AR_ACTIVITIES'
      #and table.name != 'RSEG' and table.name != 'VBPA' and table.name != 'BSAD'
      #and table.name != 'NAST'
      ):
    print("... Procesando ... : " + table.name + "   ALIAS: " + str(table.alias))
    if str(table.alias) != 'None':
      print("******** ALIAS ******** : " + str(table.alias))
    #tg_table = tg_data_model.get_tables().find(table.name)
    tg_table = tg_data_pool.get_tables().find(table.name)
    
    try:
      result_df = extract_table_from_data_pool(celonis, data_pool, data_model, table)      
      #tg_data_pool.create_table(df=result_df, table_name=table.name, drop_if_exists=True, force=True, data_source_id=tg_data_connection.id)
    except:
      result_df = pd.read_csv(f"{table.name}.csv")
      #tg_data_pool.create_table(df=result_df, table_name=table.name, drop_if_exists=True, force=True, data_source_id=tg_data_connection.id)

    tg_table.append(result_df)
    result_df.head()
    result_df = pd.DataFrame(None)
    del result_df
    #os.remove(f"{table.name}.csv")

... Procesando ... : AR_KNB5   ALIAS: KNB5
******** ALIAS ******** : KNB5


0it [00:00, ?it/s]

You might not have USE permissions


0it [00:00, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

...

0it [00:00, ?it/s]

0it [00:00, ?it/s]

... Procesando ... : AR_KNA1   ALIAS: 
******** ALIAS ******** : 


0it [00:00, ?it/s]

FileNotFoundError: ignored

# PARA PROPÓSITOS DE TESTEO DEL PQL GENERADO

In [ ]:
from pycelonis.ems import ColumnTransport, ColumnType
column_config = [
    ColumnTransport(column_name="MANDT", column_type=ColumnType.STRING, field_length=12),
    ColumnTransport(column_name="VBELN", column_type=ColumnType.STRING, field_length=40),
    ColumnTransport(column_name="POSNR", column_type=ColumnType.STRING, field_length=24),
    ColumnTransport(column_name="PARVW", column_type=ColumnType.STRING, field_length=8),
    ColumnTransport(column_name="KUNNR", column_type=ColumnType.STRING, field_length=40),
    ColumnTransport(column_name="LIFNR", column_type=ColumnType.STRING, field_length=40),
    ColumnTransport(column_name="PERNR", column_type=ColumnType.STRING, field_length=32),
    ColumnTransport(column_name="PARNR", column_type=ColumnType.STRING, field_length=40),
    ColumnTransport(column_name="ADRNR", column_type=ColumnType.STRING, field_length=40),
    ColumnTransport(column_name="ABLAD", column_type=ColumnType.STRING, field_length=100),
    ColumnTransport(column_name="XCPDK", column_type=ColumnType.STRING, field_length=4),
    ColumnTransport(column_name="HITYP", column_type=ColumnType.STRING, field_length=4),
    ColumnTransport(column_name="KNREF", column_type=ColumnType.STRING, field_length=120)
]

In [ ]:
result_df = pd.read_csv("VBPA.csv", dtype={'MANDT':'str','VBELN':'str','POSNR':'str','PARVW':'str','KUNNR':'str','LIFNR':'str','PERNR':'str','PARNR':'str','ADRNR':'str','ABLAD':'str','XCPDK':'str','HITYP':'str','KNREF':'str'})
tg_data_pool.create_table(df=result_df, table_name=table.name, drop_if_exists=True, column_config=column_config, force=True, data_source_id=tg_data_connection.id)

In [ ]:
#for table in data_model.get_tables():
#  print(table.name+' '+table.id)
table = data_model.get_table('132ba637-a897-4684-a681-ccbca8e67467') # VBPA
result_df = extract_table_from_data_pool(celonis, data_pool, data_model, table)
tg_data_pool.create_table(df=result_df, table_name=table.name, drop_if_exists=True, force=True, column_config=column_config, data_source_id=tg_data_connection.id)

In [ ]:
for table in data_model.get_tables():
  if (table.name != '_CEL_VIM_ACTIVITIES' and table.name != 'VIM_/OPT/VIM_1ITEM'):
    print("... Procesando ... : " + table.name)
    result_df = extract_table_from_data_pool(celonis, data_pool, data_model, table)
    result_df = pd.DataFrame(None)
    del result_df